In [ ]:
#originated from NLPSA_scrape_clean.ipynb

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

In [ ]:
def yf_news_scraper(yf_link, name, Num_scroll = 20):
    caps = DesiredCapabilities().CHROME
    caps["pageLoadStrategy"] = "eager"
    driver = webdriver.Chrome(ChromeDriverManager().install(), desired_capabilities=caps)
    driver.get(yf_link)
    for _ in range(Num_scroll):
        driver.execute_script("window.scrollTo(0,1000000)")
        time.sleep(2.5)
    data_raw = driver.find_elements_by_xpath("//div[@data-test-locator='mega']//a")
    print('No. of news:',len(data_raw))
    data_raw = driver.find_elements_by_xpath("//div[@data-test-locator='mega']//a")
    data_raw_topic = [i.text for i in data_raw]
#    print(data_raw_topic)
    urlss = [i.get_attribute('href') for i in data_raw]
    for url in urlss:
        if 'finance.yahoo' not in url:
            dropid = urlss.index(url)
            data_raw_topic.pop(dropid)
            urlss.pop(dropid)

    data_raw_date = []
    data_raw_content = []
    print('No. of filtered news:',len(data_raw))
    urls = urlss[:]
    print('start scraping')
    for url in urls:
        print(f'loading {url}')
        driver.get(url)
        print('finish loading pages')
        data_raw_date.append(driver.find_element_by_tag_name('time').get_attribute('datetime'))
        raw_p_content = driver.find_elements_by_xpath("//div[@class='caas-body']//p")
        print('finish finding date and content elements')

        if len(raw_p_content) < 3:
            try:
                cr_link = driver.find_element_by_link_text('Continue reading').get_attribute('href')
                data_raw_content.append(cr_link)
                continue
            except:
                pass
        content = ' '.join([i.text for i in raw_p_content])
        data_raw_content.append(content)
    print('Done scripting')

    print("Shape:",list(map(len, [data_raw_topic, data_raw_date, data_raw_content])))

    _ = [print((i,j,k[:150]+'...'),"\n") for i,j,k in zip(data_raw_date,data_raw_topic,data_raw_content)]
    driver.quit()
    print('quit driver.')
    df = pd.DataFrame({'Date' : data_raw_date, 'topic': data_raw_topic, 'content': data_raw_content, 'url':urlss})
    df.to_csv(f'raw_{name}.csv', index=False, header=True)